In [40]:
import mlflow
import mlflow.xgboost
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

In [41]:
def clean_data(dataframe1 : pd.DataFrame) :
    dataframe2 = dataframe1.copy()  
    dataframe2["smoker"] = dataframe2["smoker"].apply( lambda x : 1 if x == "yes" else 0)
    dataframe2["high_bmi"] = dataframe2["bmi"].apply( lambda x : 1 if x >30 else 0)
    dataframe2["smoker_high_bmi"] = dataframe2["bmi"]*dataframe2["smoker"]
    dataframe2 = dataframe2.drop("children", axis=1)
    dataframe2 = dataframe2.drop("sex", axis=1)
    dataframe2 = dataframe2.drop("region", axis=1)
    return dataframe2

df_initial = pd.read_csv("dataset.csv", sep=",", index_col=False)
df_modifie = clean_data(df_initial)

In [42]:
y = df_modifie['charges']
X = df_modifie.drop('charges', axis=1)

In [43]:
# Séparation des données en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [45]:
# MLflow : Définir l'expérience
mlflow.set_experiment("XGBoost_Regression_Exp")

# Lancer un run MLflow
with mlflow.start_run():
    # Initialiser et entraîner XGBRegressor
    model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, eval_metric="rmse")
    model.fit(X_train, y_train)

    # Prédictions
    y_pred = model.predict(X_test)

    # Calculer l'erreur
    rmse = root_mean_squared_error(y_test, y_pred)

    # Enregistrer les hyperparamètres
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("learning_rate", 0.1)

    # Enregistrer la métrique
    mlflow.log_metric("rmse", rmse)

    # Enregistrer le modèle
    mlflow.xgboost.log_model(model, "xgboost_regressor")

    print(f"Modèle enregistré avec RMSE={rmse:.4f}")

# Afficher l'interface MLflow
print("Lancer la commande suivante pour voir les résultats dans l'UI MLflow:")
print("mlflow ui --port 5000")

/home/nicolascassonnet/Documents/WORK/film-prediction/MLFlow/.venv/lib/python3.12/site-packages/xgboost/sklearn.py:1028: UserWarning: [11:24:25] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/04/02 11:24:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modèle enregistré avec RMSE=4757.0407
Lancer la commande suivante pour voir les résultats dans l'UI MLflow:
mlflow ui --port 5000
